In [54]:
import numpy as np
import itertools as it
from itertools import product

from scipy.optimize import dual_annealing
from wrapdisc import Objective
from wrapdisc.var import ChoiceVar

import pandas as pd

In [55]:
# Function to generate random probability distributions for bounding boxes
def generate_random_probabilities(num_boxes, num_states):
    """
    Generate random probability distributions for each bounding box.
    
    Parameters:
    - num_boxes: int, number of bounding boxes
    - num_states: int, number of states (e.g., "tree", "glass", "car", "cup")

    Returns:
    - numpy array of shape (num_boxes, num_states), where each row is a
      probability distribution over the states for a bounding box.
    """
    probabilities = np.random.rand(num_boxes, num_states)  # Generate random values
    probabilities /= probabilities.sum(axis=1, keepdims=True)  # Normalize each row to sum to 1
    return probabilities

# Generate the 4x4 relational probability matrix (for the 4 states)
def generate_symmetric_matrix(num_states):
    """Generate an n x n symmetric matrix for 0-based indexing, where n = num_states."""
    mat = np.random.rand(num_states, num_states)  # Generate random values
    mat = (mat + mat.T) / 2  # Make the matrix symmetric
    np.fill_diagonal(mat, 1)  # Fill the diagonal with 1s for self-relations
    return mat

In [56]:
def global_energy(current_states, observation, relational_matrix):
    """
    Global energy function for the entire RMRF in the current state (complete graph assumption).
    """

    energy           = 0.0
    num_vertices     = len(current_states) # Number of bounding boxes
    pairwise_weights = relational_matrix[np.ix_(current_states, current_states)] # Compute the pairwise weights by indexing the relational_matrix
    
    # 1. Factor in the observation (1-tuple) likelihood for the bounding box
    obs_probs    = observation[np.arange(num_vertices), current_states]  # Extract the probabilities for the current states
    energy      -= np.sum(np.log(obs_probs)) # Sum the log probabilities and subtract from energy
    
    # 2. Consider 2-tuples and calculate the weight as basis for the energy
    i_upper, j_upper = np.triu_indices(num_vertices, k=1)  # Get indices for all 2-tuples (upper triangle)
    energy          -= np.sum(np.log(pairwise_weights[i_upper, j_upper]))  # Subtract log of all pairwise weights

    # 3. Consider n-tuples (n >= 3) and calculate the average weight as basis for the energy
    # Iterate over all possible n-tuples, starting from n=3
    for n in range(3, num_vertices + 1):
        # Get all n-tuples (combinations of vertices of size n)
        tuples = list(it.combinations(range(num_vertices), n))
        
        # Collect pairwise weights for all tuples of this size n
        all_pairwise_weights = []
        
        for t in tuples:
            # Extract the pairwise weights for the current n-tuple (submatrix)
            submatrix = pairwise_weights[np.ix_(t, t)]  # Submatrix for the current n-tuple
            i, j = np.triu_indices(n, k=1)  # Indices for upper triangular part (pairwise combinations)
            tuple_weights = submatrix[i, j]  # Pairwise weights for this n-tuple
            
            # Add the weights to the list of all pairwise weights for this n
            all_pairwise_weights.extend(tuple_weights)
        
        # Calculate the mean over all pairwise weights across all n-tuples
        if all_pairwise_weights:  # Ensure there are weights to average
            average_weight = np.mean(all_pairwise_weights)
            energy -= np.log(average_weight)  # Subtract log of the mean of all tuple weights
    
    return energy

In [74]:
# Optimized Global Energy Function
def global_energy_optimized(current_states, observation, relational_matrix):
    energy = 0
    num_vertices = len(current_states)  # Number of bounding boxes
    pairwise_weights = relational_matrix[np.ix_(current_states, current_states)]  # Optimized submatrix extraction
    
    # 1. Factor in the observation likelihood for the bounding box (vectorized log)
    obs_probs = observation[np.arange(num_vertices), current_states]  # Extract the probabilities for the current states
    energy = -np.sum(np.log(obs_probs))  # Sum the log probabilities and subtract from energy
    
    # 2. Consider 2-tuples and calculate the weight (vectorized log of pairwise weights)
    i_upper, j_upper = np.triu_indices(num_vertices, k=1)  # Get indices for all 2-tuples (upper triangle)
    energy -= np.sum(np.log(pairwise_weights[i_upper, j_upper]))  # Subtract log of all pairwise weights
        
    return energy

In [58]:
# Function to run global_energy with all possible state combinations (optimized)
def run_global_energy_all_states(num_boxes, num_states, observation, relational_matrix):
    # Generate all possible state combinations directly as a NumPy array
    all_possible_states = np.array(list(product(range(num_states), repeat=num_boxes)))
    results = []
    
    for states in all_possible_states:
        # Pass states as a NumPy array directly
        energy = global_energy(states, observation, relational_matrix)
        results.append((states, energy))
    
    # Convert results to a DataFrame
    df_results = pd.DataFrame(results, columns=["State Combination", "Global Energy"])
    
    # Sort by global energy
    df_results = df_results.sort_values(by="Global Energy", ascending=True).reset_index(drop=True)
    
    return df_results


In [59]:
# Function to run global_energy with all possible state combinations (optimized)
def run_global_energy_all_states_optimized(num_boxes, num_states, observation, relational_matrix):
    # Generate all possible state combinations directly as a NumPy array
    all_possible_states = np.array(list(product(range(num_states), repeat=num_boxes)))
    results = []
    
    for states in all_possible_states:
        # Pass states as a NumPy array directly
        energy = global_energy_optimized(states, observation, relational_matrix)
        results.append((states, energy))
    
    # Convert results to a DataFrame
    df_results = pd.DataFrame(results, columns=["State Combination", "Global Energy"])
    
    # Sort by global energy
    df_results = df_results.sort_values(by="Global Energy", ascending=True).reset_index(drop=True)
    
    return df_results


In [60]:
def discrete_simulated_annealing(num_boxes, num_states, observation, relational_matrix, max_iter=1000, initial_temp=1.0, cooling_rate=0.99):
    # Random initial states
    current_states = np.random.choice(num_states, size=num_boxes)
    current_energy = global_energy(current_states, observation, relational_matrix)
    
    best_states = current_states.copy()
    best_energy = current_energy
    
    temperature = initial_temp
    
    for i in range(max_iter):
        # Randomly choose a box and change its state
        new_states = current_states.copy()
        random_box = np.random.randint(0, num_boxes)  # Choose a random box
        new_state = np.random.randint(0, num_states)  # Choose a new random state for that box
        new_states[random_box] = new_state
        
        # Calculate the energy of the new state configuration
        new_energy = global_energy(new_states, observation, relational_matrix)
        
        # Decide whether to accept the new state based on Metropolis criterion
        if new_energy < current_energy or np.random.rand() < np.exp((current_energy - new_energy) / temperature):
            current_states = new_states
            current_energy = new_energy
        
        # Update the best state if the new configuration is better
        if current_energy < best_energy:
            best_states = current_states
            best_energy = current_energy
        
        # Cool down the system
        temperature *= cooling_rate
    
    return best_states, best_energy

In [61]:
import numpy as np

def discrete_simulated_annealing_optimized(num_boxes, num_states, observation, relational_matrix, max_iter=1000, initial_temp=1.0, cooling_rate=0.99):
    # Random initial states
    current_states = np.random.randint(0, num_states, size=num_boxes)
    current_energy = global_energy_optimized(current_states, observation, relational_matrix)
    
    best_states = current_states.copy()  # Keep a copy of the best state
    best_energy = current_energy
    
    rand_state = np.random.default_rng()  # Use a faster random number generator
    
    # Precompute temperature updates to avoid multiplication in the loop
    temperatures = initial_temp * (cooling_rate ** np.arange(max_iter))

    for step, temperature in enumerate(temperatures):
        # Randomly choose a box and change its state (in-place update)
        random_box = rand_state.integers(0, num_boxes)  # Randomly select a box
        new_state = rand_state.integers(0, num_states)  # Generate a new random state
        
        if new_state == current_states[random_box]:
            continue  # Skip iteration if the new state is the same as the current state

        # Save old state, but avoid full array copy
        old_state = current_states[random_box]
        current_states[random_box] = new_state  # Modify the state directly
        
        # Calculate the energy of the new state configuration
        new_energy = global_energy_optimized(current_states, observation, relational_matrix)
        
        # Accept new state if it's better or by Metropolis criterion
        delta_energy = current_energy - new_energy
        if delta_energy > 0 or rand_state.random() < np.exp(delta_energy / temperature):
            current_energy = new_energy  # Accept the new state
        else:
            current_states[random_box] = old_state  # Revert the change if not accepted
        
        # Update the best state if the new configuration is better
        if current_energy < best_energy:
            best_states[:] = current_states  # Efficient in-place copy of the best state
            best_energy = current_energy
    
    return best_states, best_energy


In [75]:
# # Simulated Annealing WITH WRAPDISC

# Objective wrapper for discrete variables
def wrapped_energy_fn(wrapped_states):
    """Wrapped function to decode categorical states and compute global energy."""
    current_states = wrapped_states  # No further decoding needed for this setup
    return global_energy_optimized(current_states, observation, relational_matrix)

# Wrap the global energy function using wrapdisc
wrapped_objective = Objective(
    wrapped_energy_fn,
    variables=[
        ChoiceVar(states),  # Each bounding box can be in one of the 4 states [0, 1, 2, 3]
        ChoiceVar(states),
        ChoiceVar(states),
    ]
)


In [63]:
num_boxes = 3
num_states = 4
states = [0, 1, 2, 3]  # States corresponding to ["Glass", "Car", "Tree", "Cup"]

# Generate random current state as vecotr of length num_boxes ranging in the states
current_states = np.random.choice(states, size=num_boxes)
print("Random Current State:")
print(current_states)

# Assuming we have a 4x4 relational probability matrix for the 4 states (this should come from the ConceptNet)
relational_matrix = generate_symmetric_matrix(num_states)
print("\nRandom Relational Probability Matrix (ConceptNet):")
print(relational_matrix)

# Generate random probability distributions for 3 bounding boxes and 4 states (this should come from the Computer Vision observation)
observation = generate_random_probabilities(num_boxes=num_boxes, num_states=num_states)
print("\nRandom Bounding Box Probabilities (Computer Vision):")
print(observation, "\n")

Random Current State:
[2 0 3]

Random Relational Probability Matrix (ConceptNet):
[[1.         0.39736465 0.4884304  0.5482577 ]
 [0.39736465 1.         0.63602394 0.50768014]
 [0.4884304  0.63602394 1.         0.0366336 ]
 [0.5482577  0.50768014 0.0366336  1.        ]]

Random Bounding Box Probabilities (Computer Vision):
[[0.19862323 0.11050364 0.37327237 0.31760077]
 [0.06024503 0.4073211  0.27389927 0.2585346 ]
 [0.12873903 0.25523438 0.3499408  0.26608579]] 



In [64]:
# Run global_energy for all possible state combinations and get the sorted results
df_all_states_results = run_global_energy_all_states(
    num_boxes=num_boxes,
    num_states=num_states,
    observation=observation,
    relational_matrix=relational_matrix
)

# Display the DataFrame with state combinations and global energy values
print(df_all_states_results)

   State Combination  Global Energy
0          [2, 2, 2]       3.330433
1          [3, 3, 3]       3.823622
2          [2, 1, 2]       4.116561
3          [2, 1, 1]       4.432142
4          [1, 1, 1]       4.466433
..               ...            ...
59         [3, 2, 2]      11.133430
60         [3, 3, 2]      11.191161
61         [2, 2, 3]      11.245862
62         [2, 3, 3]      11.303593
63         [3, 2, 3]      11.407375

[64 rows x 2 columns]


In [65]:
# Run global_energy for all possible state combinations and get the sorted results
df_all_states_results = run_global_energy_all_states_optimized(
    num_boxes=num_boxes,
    num_states=num_states,
    observation=observation,
    relational_matrix=relational_matrix
)

# Display the DataFrame with state combinations and global energy values
print(df_all_states_results)

   State Combination  Global Energy
0          [2, 2, 2]       3.330433
1          [3, 3, 3]       3.823622
2          [2, 1, 2]       3.838630
3          [2, 1, 1]       4.154212
4          [1, 1, 1]       4.466433
..               ...            ...
59         [3, 2, 2]      10.105525
60         [3, 3, 2]      10.163256
61         [2, 2, 3]      10.217957
62         [2, 3, 3]      10.275688
63         [3, 2, 3]      10.379471

[64 rows x 2 columns]


In [66]:
n_runs = 100
# Function to run the simulated annealing multiple times and collect results
def run_simulated_annealing_multiple_times(n_runs, num_boxes, num_states, observation, relational_matrix):
    results = {}
    
    for _ in range(n_runs):
        optimized_states, minimized_energy = discrete_simulated_annealing(
            num_boxes=num_boxes, 
            num_states=num_states, 
            observation=observation, 
            relational_matrix=relational_matrix,
            max_iter=1000,
            initial_temp=1.0,
            cooling_rate=0.99
        )
        
        # Convert list to a tuple to use as a dictionary key
        state_tuple = tuple(optimized_states)
        
        # If the state is already in results, update the count and energy
        if state_tuple in results:
            results[state_tuple]['count'] += 1
        else:
            results[state_tuple] = {'minimized_energy': minimized_energy, 'count': 1}
    
    # Convert results to a DataFrame
    df_results = pd.DataFrame([
        {'Optimized States': states, 'Minimized Global Energy': data['minimized_energy'], 'Count': data['count']}
        for states, data in results.items()
    ])
    
    # Sort by minimized energy
    df_results = df_results.sort_values(by="Minimized Global Energy", ascending=True).reset_index(drop=True)
    
    return df_results

# Run the simulation and get the ordered results
df_ordered_results = run_simulated_annealing_multiple_times(
    n_runs=n_runs,
    num_boxes=num_boxes,
    num_states=num_states,
    observation=observation,
    relational_matrix=relational_matrix
)

print(df_ordered_results)

  Optimized States  Minimized Global Energy  Count
0        (2, 2, 2)                 3.330433     68
1        (3, 3, 3)                 3.823622     32


In [71]:
n_runs = 100
# Function to run the simulated annealing multiple times and collect results
def run_simulated_annealing_multiple_times_optimized(n_runs, num_boxes, num_states, observation, relational_matrix):
    results = {}
    
    for _ in range(n_runs):
        optimized_states, minimized_energy = discrete_simulated_annealing_optimized(
            num_boxes=num_boxes, 
            num_states=num_states, 
            observation=observation, 
            relational_matrix=relational_matrix,
            max_iter=1000,
            initial_temp=1.0,
            cooling_rate=0.99
        )
        
        # Convert list to a tuple to use as a dictionary key
        state_tuple = tuple(optimized_states)
        
        # If the state is already in results, update the count and energy
        if state_tuple in results:
            results[state_tuple]['count'] += 1
        else:
            results[state_tuple] = {'minimized_energy': minimized_energy, 'count': 1}
    
    # Convert results to a DataFrame
    df_results = pd.DataFrame([
        {'Optimized States': states, 'Minimized Global Energy': data['minimized_energy'], 'Count': data['count']}
        for states, data in results.items()
    ])
    
    # Sort by minimized energy
    df_results = df_results.sort_values(by="Minimized Global Energy", ascending=True).reset_index(drop=True)
    
    return df_results

# Run the simulation and get the ordered results
df_ordered_results = run_simulated_annealing_multiple_times_optimized(
    n_runs=n_runs,
    num_boxes=num_boxes,
    num_states=num_states,
    observation=observation,
    relational_matrix=relational_matrix
)

print(df_ordered_results)

  Optimized States  Minimized Global Energy  Count
0        (2, 2, 2)                 3.330433     77
1        (3, 3, 3)                 3.823622     23


In [76]:
# Get the bounds from the wrapped objective
bounds = wrapped_objective.bounds

# Use dual annealing to minimize the wrapped energy function
result = dual_annealing(wrapped_objective, bounds=bounds, seed=42, maxiter=10000)

# Decode the solution back into the categorical form
decoded_solution = wrapped_objective.decode(result.x)
print("Optimized States:", decoded_solution)
print("Minimized Global Energy:", result.fun)
print("Cause of the termination:", result.message)

Optimized States: (2, 2, 2)
Minimized Global Energy: 3.330433079492188
Cause of the termination: ['Maximum number of iteration reached']


# Comparing to probability functions

In [77]:
# Optimized Global Probability Function (Multiplicative Version)
def global_probability_optimized(current_states, observation, relational_matrix):
    probability = 1  # Start with 1 for multiplication
    num_vertices = len(current_states)  # Number of bounding boxes
    pairwise_weights = relational_matrix[np.ix_(current_states, current_states)]  # Optimized submatrix extraction
    
    # 1. Factor in the observation likelihood for the bounding box
    obs_probs = observation[np.arange(num_vertices), current_states]  # Extract the probabilities for the current states
    probability *= np.prod(obs_probs)  # Multiply the observation probabilities
    
    # 2. Consider 2-tuples and calculate the weight (direct pairwise weights)
    i_upper, j_upper = np.triu_indices(num_vertices, k=1)  # Get indices for all 2-tuples (upper triangle)
    probability *= np.prod(pairwise_weights[i_upper, j_upper])  # Multiply all pairwise weights
    
    return probability

In [78]:
# Function to run global_probability with all possible state combinations (optimized)
def run_global_probability_all_states_optimized(num_boxes, num_states, observation, relational_matrix):
    # Generate all possible state combinations directly as a NumPy array
    all_possible_states = np.array(list(product(range(num_states), repeat=num_boxes)))
    results = []
    
    for states in all_possible_states:
        # Pass states as a NumPy array directly
        probability = global_probability_optimized(states, observation, relational_matrix)
        results.append((states, probability))
    
    # Convert results to a DataFrame
    df_results = pd.DataFrame(results, columns=["State Combination", "Global Probability"])
    
    # Sort by global probability in descending order
    df_results = df_results.sort_values(by="Global Probability", ascending=False).reset_index(drop=True)
    
    return df_results

In [79]:
# Run global_probability for all possible state combinations and get the sorted results
df_all_states_results = run_global_probability_all_states_optimized(
    num_boxes=num_boxes,
    num_states=num_states,
    observation=observation,
    relational_matrix=relational_matrix
)

# Display the DataFrame with state combinations and global probability values
print(df_all_states_results)

   State Combination  Global Probability
0          [2, 2, 2]            0.035778
1          [3, 3, 3]            0.021849
2          [2, 1, 2]            0.021523
3          [2, 1, 1]            0.015698
4          [1, 1, 1]            0.011488
..               ...                 ...
59         [3, 2, 2]            0.000041
60         [3, 3, 2]            0.000039
61         [2, 2, 3]            0.000037
62         [2, 3, 3]            0.000034
63         [3, 2, 3]            0.000031

[64 rows x 2 columns]


In [81]:
# Run the simulation and get the ordered results
df_ordered_results = run_simulated_annealing_multiple_times_optimized(
    n_runs=n_runs,
    num_boxes=num_boxes,
    num_states=num_states,
    observation=observation,
    relational_matrix=relational_matrix
)

print(df_ordered_results)

  Optimized States  Minimized Global Energy  Count
0        (2, 2, 2)                 3.330433     86
1        (3, 3, 3)                 3.823622     14


🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉